# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from newsapi import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

In [ ]:
# Read your api key environment variable
load_dotenv()
news_api_key = os.getenv("NEWS_API_KEY")

In [ ]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=news_api_key)


In [ ]:
# Define timeframe for grabbing articles
today = pd.Timestamp.now()
previous = today - pd.Timedelta(4, unit="W")
TIME_FORMAT_STRING = "%Y-%m-%d"


In [ ]:
# Fetch the Bitcoin news articles
bitcoin_headlines = newsapi.get_everything(q='bitcoin',
                                      language='en',
                                      sort_by='relevancy')

# Print total and show sample
print(f"Total articles about bitcoin: {bitcoin_headlines['totalResults']}")
bitcoin_headlines["articles"][0]


In [ ]:
# Fetch the Ethereum news articles
ethereum_headlines = newsapi.get_everything(q='Ethereum',
                                      language='en',
                                      sort_by='relevancy')

# Print total and show sample
print(f"Total articles about ethereum: {ethereum_headlines['totalResults']}")
ethereum_headlines["articles"][0]


In [ ]:
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)


In [ ]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiments = []

for article in bitcoin_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        normalized_compound_score = get_sentiment(sentiment["compound"])
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "normalized_compound_score": normalized_compound_score,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass

bitcoin_df = pd.DataFrame(bitcoin_sentiments)
cols = ["date", "text", "compound", "normalized_compound_score", "positive", "negative", "neutral"]
bitcoin_df = bitcoin_df[cols]
bitcoin_df.head()


In [ ]:
# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result


In [ ]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiments = []

for article in ethereum_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        normalized_compound_score = get_sentiment(sentiment["compound"])
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        ethereum_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "normalized_compound_score": normalized_compound_score,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass

ethereum_df = pd.DataFrame(ethereum_sentiments)
cols = ["date", "text", "compound", "normalized_compound_score", "positive", "negative", "neutral"]
ethereum_df = ethereum_df[cols]
ethereum_df.head()


In [ ]:
# Download/Update the VADER Lexicon
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Describe the Bitcoin Sentiment
bitcoin_df.describe()


In [ ]:
# Describe the Ethereum Sentiment
ethereum_df.describe()


In [ ]:
# Generate plots
bitcoin_df.plot(
    y=["compound", "normalized_compound_score"],
    kind="bar",
    title="News Title and Text Sentiment Comparisson",
    figsize=(10, 8),
    grid=True
)


In [ ]:
ethereum_df.plot(
    y=["compound", "normalized_compound_score"],
    kind="bar",
    title="News Title and Text Sentiment Comparisson",
    figsize=(10, 8),
    grid=True
)


In [ ]:
plt = ethereum_df.plot(
    x="date",
    y="compound",
    kind="line",
    title="News Title and Text Sentiment Comparisson",
    figsize=(20, 20),
    grid=True
)

bitcoin_df.plot(
    x="date",
    y="compound",
    kind="line",
    title="News Title and Text Sentiment Comparisson",
    figsize=(10, 8),
    grid=True,
    ax=plt
)


### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum has a higher mean positive score (0.060950) compared to that of Bitcoin (0.053526). This implies that articles have a generally more positive sentiment for ethereum than bitcoin.

Q: Which coin had the highest compound score?

A: Bitcoin has a higher compound score (0.205768) than ethereum (0.122565). Overall, Bitcoin has a higher level of confidence.

Q. Which coin had the highest positive score?

A: Ethereum has a higher maximum positive score (0.318000) than bitcoin (0.174000). This might imply that there is more excitement about ethereum.

---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:

1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Expand the default stopwords list if necessary
english_stopwords = set(stopwords.words('english'))
custom_stopwords = set(['said', 'sent', 'found', 'including', 'today', 'announced', 'week', 'basically', 'also', ',', 'weekly', 'bitcoin', 'ethereum', 'following', 'char', 'ha' ])

full_stopwords = english_stopwords.union(custom_stopwords)


In [ ]:
"[" + punctuation + "]"

In [ ]:
regex = re.compile("[" + punctuation + "]")
word = "="
print(re.match("[" + punctuation + "]+", word) is not None)

In [ ]:
# Complete the tokenizer function
regex_punctuation_match_str = ".*[" + punctuation + "]+.*"
lemmatizer = WordNetLemmatizer()

def tokenizer(text):
    """Tokenizes text."""
    
    # Create a list of the words
    words = word_tokenize(text)

    # Convert the words to lowercase
    words = [word.lower() for word in words]

    # Remove the punctuation
    words = [word.lower() for word in words if re.match(regex_punctuation_match_str, word) is None]

    # regex = re.compile("[^a-zA-Z ]")
    # text = [regex.sub('', word) for word in text]

    # punctuation
    # regex = re.compile("[" + punctuation + "]")
    # re_clean = regex.sub('', article)
    

    # Lemmatize Words into root words
    words = [lemmatizer.lemmatize(word) for word in words]


    # Remove the stop words
    words = [word.lower() for word in words if word.lower() not in full_stopwords]

    return words


In [ ]:
# an_array = []
# for text in bitcoin_df["text"].values:
#     an_array.append(tokenizer(text))

In [ ]:
# Create a new tokens column for bitcoin
# bitcoin_df["tokens"] = []
# tokenizer(bitcoin_df["text"])

bitcoin_df_tokens = []
for text in bitcoin_df["text"].values:
    bitcoin_df_tokens.append(tokenizer(text))

bitcoin_df["tokens"] = bitcoin_df_tokens
bitcoin_df.head()


In [ ]:
# Create a new tokens column for ethereum
ethereum_df_tokens = []
for text in ethereum_df["text"].values:
    ethereum_df_tokens.append(tokenizer(text))

ethereum_df["tokens"] = ethereum_df_tokens
ethereum_df.head()


---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
bitcoin_ngrams = [ Counter(ngrams(token, n=2)) for token in bitcoin_df["tokens"].values ]
bitcoin_ngrams[0]


In [ ]:
# Generate the Ethereum N-grams where N=2
ethereum_ngrams = [ Counter(ngrams(token, n=2)) for token in ethereum_df["tokens"].values ]
ethereum_ngrams[0]


In [ ]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)


In [ ]:
# Get the top 10 words for Bitcoin
all_words_bitcoin = []
for token in bitcoin_df.tokens:
    all_words_bitcoin += token

token_count(all_words_bitcoin, 10)

# bitcoin_df["tokens"].values
# token_count(bitcoin_df["tokens"].values, 10)


In [ ]:
all_words_bitcoin_str = ' '.join(all_words_bitcoin)
all_words_bitcoin_str

In [ ]:
# Get the top 10 words for Ethereum
all_words_ethereum = []
for token in ethereum_df.tokens:
    all_words_ethereum += token

token_count(all_words_ethereum, 10)


# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
all_words_bitcoin_str = ' '.join(all_words_bitcoin)
num_bitcoin_words = 30
wc = WordCloud(width=1200, height=800, max_words=num_bitcoin_words).generate(all_words_bitcoin_str)
plt.title(f"Top {num_bitcoin_words} Words in Articles about Bitcoin", fontsize=25, fontweight="bold")
plt.imshow(wc)


In [ ]:
# Generate the Ethereum word cloud
all_words_ethereum_str = ' '.join(all_words_ethereum)
num_ethereum_words = 30
wc = WordCloud(width=1200, height=800, max_words=num_ethereum_words).generate(all_words_ethereum_str)
plt.title(f"Top {num_ethereum_words} Words in Articles about Ethereum", fontsize=25, fontweight="bold")
plt.imshow(wc)


# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Optional - download a language model for SpaCy
!python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [ ]:
# Concatenate all of the bitcoin text together
bitcoin_corpus = ' '.join(bitcoin_df["text"].values)


In [ ]:
# Run the NER processor on all of the text
btc_ner = nlp(bitcoin_corpus)

# Add a title to the document
btc_ner.user_data["title"] = "All the Article Text on Bitcoin"


In [ ]:
# Render the visualization
displacy.render(btc_ner, style='ent')


In [ ]:
# List all Entities
bitcoin_entities_list = [ent.text for ent in btc_ner.ents]
bitcoin_entities_set = set(bitcoin_entities_list)
bitcoin_entities_list[0:10]


---

## Ethereum NER

In [ ]:
# Concatenate all of the ethereum text together
ethereum_corpus = ' '.join(ethereum_df["text"].values[0:50])


In [ ]:
# Run the NER processor on all of the text
eth_ner = nlp(ethereum_corpus)

# Add a title to the document
eth_ner.user_data["title"] = "All the Article Text on Ethereum"


In [ ]:
# Render the visualization
colors = {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
options = {"ents": ["ORG"], "colors": colors}
displacy.render(eth_ner, style="ent", options=options)


In [ ]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro"}

displacy.serve(eth_ner, style="dep", options=options)


In [ ]:
nlp = spacy.load("en_core_web_sm")

def displacy_service(text):
    doc = nlp(text)
    return displacy.parse_deps(doc)


In [ ]:
# List all Entities
ethereum_entities_list = [ent.text for ent in eth_ner.ents]
ethereum_entities_set = set(ethereum_entities_list)
ethereum_entities_list[0:10]
